In [124]:
from skimage import external
from IPython.display import display
import japanize_matplotlib
from matplotlib import pyplot
pyplot.style.use('kotaro')
import pandas as pd
import scipy as sc
import numpy as np
import os
import imageio
import zipfile

srcRoot = ("src/10_4231_R7RX991C")
srcRoot2 = ("src/TokumaShimizu")

In [125]:
with zipfile.ZipFile('src/10_4231_R7RX991C.zip') as existing_zip:
    existing_zip.extract('10_4231_R7RX991C/bundle.zip','src')
    existing_zip.extract('10_4231_R7RX991C/documentation/Site3_Project_and_Ground_Reference_Files.zip','src')

with zipfile.ZipFile('src/10_4231_R7RX991C/bundle.zip') as existing_zip:
    existing_zip.extract('aviris_hyperspectral_data/19920612_AVIRIS_IndianPine_Site3.tif', srcRoot)
!rm -Rf src/10_4231_R7RX991C/bundle.zip

with zipfile.ZipFile('src/10_4231_R7RX991C/documentation/Site3_Project_and_Ground_Reference_Files.zip') as existing_zip:
    existing_zip.extract('Site3_Project_and_Ground_Reference_Files/19920612_AVIRIS_IndianPine_Site3_gr.tif',srcRoot)
!rm -Rf src/10_4231_R7RX991C/documentation


In [126]:
pd.options.display.precision = 2
# Hyper 220bands
#=========
hypTif =  ("aviris_hyperspectral_data/19920612_AVIRIS_IndianPine_Site3.tif") 
srcimg = external.tifffile.imread(os.path.join(srcRoot,hypTif)) # (bands,w,h)
srcimg = srcimg.transpose(1,2,0) # (w,h,bands)
srcimg = srcimg.reshape(-1,220) # (w*h, bands)
columns=[]
for s in range(220):
    columns = columns + ["#{}".format(s)]
feature_df = pd.DataFrame(srcimg, columns=columns)
display(feature_df.describe())

,#0,#1,#2,#3,#4,#5,#6,#7,#8,#9,...,#210,#211,#212,#213,#214,#215,#216,#217,#218,#219
count,21025.00,21025.00,21025.00,21025.00,21025.00,21025.00,21025.00,21025.00,21025.00,21025.00,...,21025.00,21025.00,21025.00,21025.00,21025.00,21025.00,21025.00,21025.00,21025.00,21025.00
mean,3341.31,4091.96,4277.52,4170.08,4516.69,4790.61,4848.33,4714.74,4668.92,4439.26,...,1061.96,1060.24,1063.30,1050.07,1040.23,1043.39,1030.22,1015.60,1008.51,1006.79
std,212.25,227.89,257.82,280.38,346.04,414.39,469.24,491.73,533.23,539.49,...,38.82,38.69,41.91,34.23,26.45,29.79,20.87,11.44,7.05,7.00
min,2632.00,3477.00,3649.00,3578.00,3840.00,4056.00,4004.00,3865.00,3775.00,3560.00,...,998.00,999.00,999.00,993.00,990.00,992.00,989.00,986.00,981.00,980.00
25%,3179.00,3889.00,4066.00,3954.00,4214.00,4425.00,4421.00,4263.00,4173.00,3940.00,...,1026.00,1024.00,1024.00,1019.00,1016.00,1016.00,1012.00,1006.00,1004.00,1003.00
50%,3343.00,4107.00,4237.00,4126.00,4478.00,4754.00,4808.00,4666.00,4632.00,4404.00,...,1053.00,1052.00,1054.00,1043.00,1033.00,1037.00,1026.00,1014.00,1009.00,1005.00
75%,3510.00,4247.00,4479.00,4350.00,4772.00,5093.00,5198.00,5100.00,5084.00,4860.00,...,1099.00,1098.00,1104.00,1083.00,1066.00,1072.00,1050.00,1024.00,1014.00,1010.00
max,4536.00,5744.00,6361.00,6362.00,7153.00,7980.00,8284.00,8128.00,8194.00,7928.00,...,1287.00,1289.00,1315.00,1258.00,1201.00,1245.00,1167.00,1076.00,1037.00,1034.00


In [127]:
from matplotlib import colors

# Target RGB
#==============
grp17Tif = ("Site3_Project_and_Ground_Reference_Files/19920612_AVIRIS_IndianPine_Site3_gr.tif")
targetimg17 = external.tifffile.imread(os.path.join(srcRoot,grp17Tif)) # (w,h,bands)
targetimg17 = targetimg17.reshape(-1,3) # (w*h,bands)
targetimg17 = np.uint8(targetimg17 /256) # uint16->uint8
targethex17 = pd.Series()
for i in range(0,21025):
    #print("({r},{g},{b})".format(r=targetimg17[i,0],g=targetimg17[i,1],b=targetimg17[i,2]))
    targethex17 = targethex17.append(pd.Series(format(targetimg17[i,0]<<16|targetimg17[i,1]<<8|targetimg17[i,2],'06x')))

# Label number
#=========
labelpgm = ("true.pgm")
pgmlabels = pd.read_csv(os.path.join(srcRoot2,labelpgm), delim_whitespace=True, header=None, skiprows=3)
labels = pd.Series(np.ravel(pgmlabels.values),name='Categolies') # or pgmlabels.values.reshape(-1,1), reshape to 1-columns  
labels = labels.dropna() # delete last few Nan samples

# Label names
#========
## 17 classes
cat17 = ['BackGround','Alfalfa','Corn-notill','Corn-min','Corn',
                                                 'Grass/Pasture','Grass/Trees','Grass/pasture-mowed',
                                                 'Hay-windrowed','Oats','Soybeans-notill','Soybeans-min',
                                                 'Soybean-clean','Wheat','Woods','Bldg-Grass-Tree-Drives',
                                                 'Stone-steel towers']
dict17_for17 = {0.0:'BackGround', 1.0:'Alfalfa', 2.0:'Corn-notill', 3.0:'Corn-min', 4.0:'Corn',
                        5.0:'Grass/Pasture', 6.0:'Grass/Trees', 7.0:'Grass/pasture-mowed',
                        8.0:'Hay-windrowed', 9.0:'Oats', 10.0:'Soybeans-notill', 11.0:'Soybeans-min', 12.0:'Soybean-clean',
                        13.0:'Wheat', 14.0:'Woods', 15.0:'Bldg-Grass-Tree-Drives', 16.0:'Stone-steel towers'}
labels17 = labels.map(dict17_for17)


In [128]:
# label vs RGBmap
#===================
## 17 classes
RGBvslabel17 = pd.DataFrame(np.array([labels17.values,targethex17.values]).T,columns=['label17','hex'])
RGBvslabel17['label17'] = RGBvslabel17['label17'].astype("category",categories=cat17, ordered=False)
display(pd.DataFrame(RGBvslabel17['label17'].value_counts()))

RGBvslabels17 = pd.concat([
    RGBvslabel17[RGBvslabel17['label17']=='BackGround'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Alfalfa'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Corn-notill'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Corn-min'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Corn'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Grass/Pasture'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Grass/Trees'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Grass/pasture-mowed'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Hay-windrowed'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Oats'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Soybeans-notill'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Soybeans-min'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Soybean-clean'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Wheat'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Woods'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Bldg-Grass-Tree-Drives'].head(1),
    RGBvslabel17[RGBvslabel17['label17']=='Stone-steel towers'].head(1)
],axis=0)

hex17 = RGBvslabels17['hex'].values
RGBvslabel17['hex'] = RGBvslabel17['hex'].astype("category",categories=hex17, ordered=False)



,label17
BackGround,10659
Soybeans-min,2468
Corn-notill,1434
Woods,1294
Soybeans-notill,968
Corn-min,834
Grass/Trees,747
Soybean-clean,614
Grass/Pasture,497
Hay-windrowed,489


In [133]:
cordinate_df = pd.DataFrame([(x, y) for x in range(0,145) for y in range(0,145)],
                            columns=['column#','line#'])

#cordinate_df = cordinate_df.set_index(['column#','line#'])

labels17 = pd.Series(labels17).astype('category',categories=cat17,ordered=False)
labels17.name = 'categories#'
labels10 = pd.Series(labels10).astype('category',categories=cat10,ordered=False)
labels10.name = 'categories#'

file_name = 'data/IndianPines'

file_name = file_name+'.csv'
data = pd.concat([cordinate_df,feature_df],axis=1)
df = pd.concat([data,labels17],axis=1)
target_names = cat17
for i,v in enumerate(hex17):
    hex_names[i] = '#{code}'.format(code=v)

data = df.loc[:,df.columns!='categories#']
feature_names = data.columns
target = df[['column#','line#','categories#']] 
target['categories#'] = target['categories#'].cat.codes
n_samples = data.shape[0]
n_features = data.shape[1]

display(data)
display(feature_names)
display(target)
display(target_names)
display(hex_names)
display(n_samples)
display(n_features)

df1 = pd.DataFrame([[n_samples, n_features]])
df1.to_csv(file_name,header=False,index=False)
df2 = pd.DataFrame([target_names])
df2.to_csv(file_name,header=False,index=False,mode='a')
df3 = pd.DataFrame([hex_names])
df3.to_csv(file_name,header=False,index=False,mode='a')
df4 = pd.DataFrame([feature_names])
df4.to_csv(file_name,header=False,index=False,mode='a')
df5 = df
df5['categories#'] = df5['categories#'].cat.codes
df5.to_csv(file_name,header=False,index=False,mode='a')

        




/home/kotaro/work/practice/python/IndianPines/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,column#,line#,#0,#1,#2,#3,#4,#5,#6,#7,...,#210,#211,#212,#213,#214,#215,#216,#217,#218,#219
0,0,0,3172,4142,4506,4279,4782,5048,5213,5106,...,1094,1090,1112,1090,1062,1069,1057,1020,1020,1005
1,0,1,3348,4266,4502,4426,4853,5249,5352,5353,...,1108,1104,1117,1091,1079,1085,1064,1029,1020,1000
2,0,2,3687,4266,4421,4498,5019,5293,5438,5427,...,1111,1114,1114,1100,1065,1092,1061,1030,1016,1009
3,0,3,3517,4258,4603,4493,4958,5234,5417,5355,...,1122,1108,1109,1109,1071,1088,1060,1030,1006,1005
4,0,4,3514,4018,4675,4417,4886,5117,5215,5096,...,1110,1107,1112,1094,1072,1087,1052,1034,1019,1019
5,0,5,3343,4262,4320,4337,4696,4911,4957,4900,...,1092,1095,1105,1083,1063,1079,1060,1034,1011,1005
6,0,6,3343,4134,4333,4198,4482,4833,4962,4657,...,1096,1093,1096,1074,1066,1069,1060,1025,1015,1019
7,0,7,3343,4014,4590,4284,4663,5038,5026,4921,...,1108,1097,1110,1081,1071,1077,1052,1034,1015,1015
8,0,8,3346,4142,4413,4488,4777,5136,5192,5113,...,1115,1105,1117,1098,1072,1082,1055,1034,1011,1015
9,0,9,3522,4273,4417,4341,4782,5112,5144,5062,...,1112,1103,1110,1093,1076,1091,1060,1030,1024,1010


Index(['column#', 'line#', '#0', '#1', '#2', '#3', '#4', '#5', '#6', '#7',
       ...
       '#210', '#211', '#212', '#213', '#214', '#215', '#216', '#217', '#218',
       '#219'],
      dtype='object', length=222)

,column#,line#,categories#
0,0,0,3
1,0,1,3
2,0,2,3
3,0,3,3
4,0,4,3
5,0,5,3
6,0,6,3
7,0,7,3
8,0,8,3
9,0,9,3


['BackGround',
 'Alfalfa',
 'Corn-notill',
 'Corn-min',
 'Corn',
 'Grass/Pasture',
 'Grass/Trees',
 'Grass/pasture-mowed',
 'Hay-windrowed',
 'Oats',
 'Soybeans-notill',
 'Soybeans-min',
 'Soybean-clean',
 'Wheat',
 'Woods',
 'Bldg-Grass-Tree-Drives',
 'Stone-steel towers']

array(['#ffffff', '#fffe89', '#031cf1', '#ff5901', '#05ff85', '#ff02fb',
       '#5901ff', '#03abff', '#0cff07', '#acaf54', '#a04e9e', '#65adff',
       '#3c5b70', '#68c03f', '#8b452e', '#77ffac', '#feff03'],
      dtype=object)

21025

222